# Dev

> TODO fill in description

In [ ]:
#| default_exp dev

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from pathlib import Path
import pickle

import fbdev
from fbdev.utils import is_mutually_exclusive, get_git_root_directory
from fbdev.packet import PacketRegistry, PacketMovement
from fbdev.port import PortType
from fbdev.component import PortType
from fbdev.graph import Graph
from fbdev.node import Node
from fbdev.runtime import BatchExecutor

In [ ]:
#|hide
show_doc(fbdev.dev.enter_node_execution)

/Users/lukastk/miniconda3/envs/fbdev/lib/python3.10/site-packages/fastcore/docscrape.py:230: UserWarning: Unknown section Parameters:
  else: warn(msg)
/Users/lukastk/miniconda3/envs/fbdev/lib/python3.10/site-packages/fastcore/docscrape.py:230: UserWarning: Unknown section Returns:
  else: warn(msg)
/Users/lukastk/miniconda3/envs/fbdev/lib/python3.10/site-packages/fastcore/docscrape.py:230: UserWarning: Unknown section Raises:
  else: warn(msg)
/Users/lukastk/miniconda3/envs/fbdev/lib/python3.10/site-packages/fastcore/docscrape.py:230: UserWarning: Unknown section Notes:
  else: warn(msg)
/Users/lukastk/miniconda3/envs/fbdev/lib/python3.10/site-packages/fastcore/docscrape.py:230: UserWarning: Unknown section Example:
  else: warn(msg)


---

### enter_node_execution

>      enter_node_execution (graph:fbdev.graph.Graph, node_address:str,
>                            repo_path:str=None, cache_path:str=None,
>                            packet_registry:fbdev.packet.PacketRegistry=None)

*Simulates the process of a component in a graph using historical packet data.

This function sets up a simulated environment for a specific node in the graph,
allowing it to process inputs based on a cached packet registry. It's useful for
testing and debugging node behavior without running the entire system.

The process will keep running indefinitely, until explicitly cancelled.*

The function does not need to be async. However, `nbdev` will not treat a cell as running within an event loop if there is no async-specific syntax in it. That is,
if `enter_node_execution` was not async, then running a cell like

```python
self = enter_node_execution(graph=g, node_address='>net>node1')
```

will throw an exception during `nbdev_prepare`. However

```python
self = await enter_node_execution(graph=g, node_address='>net>node1')
```

will work fine.

In [ ]:
#|export
async def enter_node_execution(
        graph:Graph,
        node_address:str,
        repo_path:str=None,
        cache_path:str=None,
        packet_registry:PacketRegistry=None
    ):
    """
    Simulates the process of a component in a graph using historical packet data.

    This function sets up a simulated environment for a specific node in the graph,
    allowing it to process inputs based on a cached packet registry. It's useful for
    testing and debugging node behavior without running the entire system.
    
    The process will keep running indefinitely, until explicitly cancelled.

    Parameters:
    -----------
    graph : Graph
        The graph object containing the node to be simulated.
    node_address : str
        The address of the node in the graph to be simulated.
    repo_path : str, optional
        The path to the git repository root. If None, the function will attempt to
        find the git root directory automatically.
    cache_path : str, optional
        The path to the cached packet registry file. If None, it will be constructed
        based on the repo_path.
    packet_registry : PacketRegistry, optional
        A pre-loaded PacketRegistry object. If provided, cache_path and repo_path
        will be ignored.

    Returns:
    --------
    None
        The function doesn't return a value but sets up and runs the simulation asynchronously.

    Raises:
    -------
    ValueError
        If repo_path, cache_path, and packet_registry are not mutually exclusive.
        If the git repository root cannot be found when repo_path is None.
    FileNotFoundError
        If the specified cache file does not exist.

    Notes:
    ------
    - The function uses a cached packet registry to simulate inputs to the node.
    - The node's is_passive attribute is set to True to prevent actual execution.
    - The simulation runs asynchronously, processing historical packet movements.
    - Output from the node is received but not processed further.

    Example:
    --------
    >>> graph = Graph(...)  # Your graph object
    >>> node_address = "example.node"
    >>> get_simulated_component_process(graph, node_address)
    """
    
    if not is_mutually_exclusive(repo_path, cache_path, packet_registry):
        raise ValueError("repo_path, cache_path and packet_registry are mutually exclusive.")
    
    if repo_path is None and cache_path is None:
        repo_path = get_git_root_directory()
        if repo_path is None:
            raise ValueError("Cannot find root path of repository. Either provide the repo_path, cache_path, or initialise the repository using git.")
    if cache_path is None:
        cache_path = Path(repo_path) / '.fbdev' / 'packet_registry_cache.pkl'
        
    if not cache_path.exists():
        raise FileNotFoundError(f"Cannot find cache at '{cache_path}")
    
    if cache_path is not None:
        packet_registry = pickle.load(open(cache_path, 'rb'))
        
    node_spec = graph.get_by_address(node_address)
    node = Node(node_spec)
    node.initialise()
    node.component_process.is_passive = True # A hack to ensure that the actual execute method is not called.
    
    async def input_sender(port_type, port_name, packet):
        data = await packet_registry._peek(packet)
        await node.send_input(port_type, port_name, data)
        
    output = {}
    async def output_receiver(port_type, port_name):
        while True:
            data = await node.receive_output(port_type, port_name)
            # Do nothing with the data 
    
    input_senders = []
    for act in packet_registry.history:
        if type(act) == PacketMovement:
            if act.dest == node_address:
                packet = packet_registry.packets[act.packet_id]
                input_senders.append(input_sender(act.port_type, act.port_name, packet))
    output_receivers = [output_receiver(PortType.OUTPUT, port_name) for port_name in node.port_specs.output.keys()]
    
    async def main():
        node.run()
        await node._execute_with_exception_monitoring(*input_senders, *output_receivers)
        await node._execute_with_exception_monitoring(node.async_stop())
        
    asyncio.create_task(main())
    
    return node.component_process

In [ ]:
#|hide
show_doc(fbdev.dev.save_graph_execution_cache)

---

### save_graph_execution_cache

>      save_graph_execution_cache (g, packet_registry_path=None)

In [ ]:
#|export
async def save_graph_execution_cache(g, packet_registry_path=None):
    if packet_registry_path is None:
        repo_path = get_git_root_directory()
        packet_registry_path = Path(repo_path) / '.fbdev' / 'packet_registry_cache.pkl'
        
    async with BatchExecutor.from_graph(g) as runtime:
        await runtime.async_execute(1)
        packet_registry = runtime._net.packet_registry
        path = Path(fbdev.dev.get_git_root_directory(), '.fbdev')
        path.mkdir(parents=True, exist_ok=True)
        pickle.dump(packet_registry, open(path / 'packet_registry_cache.pkl', 'wb'))